## 합성곱과 풀링(Convolution and Pooling)
합성곱 신경망(CNN)은 이미지 처리에 탁월한 성능을 보이는 신경망입니다. 이번 챕터에서는 합성곱 신경망에 대해서 학습합니다.

합성곱 신경망은 크게 `합성곱층(Convolution Layer)`와 `풀링층(Pooling Layer)`로 구성됩니다. 아래의 그림은 합성곱 신경망의 일반적인 예를 보여줍니다.
![](../Static/convpooling.png)   

위의 그림에서 `CONV`는 합성곱 연산을 의미하고, 합성곱 연산의 결과가 활성화 함수 ReLU를 지납니다. 이 두 과정을 합성곱층이라고 합니다. 그 후에 `POOL`이라는 구간을 지나는데 이는 풀링 연산을 의미하며 풀링층이라고 합니다.

이번 챕터에서는 합성곱 연산과 풀링 연산의 의미에 대해서 학습합니다.

## 합성곱 신경망의 대두
합성곱 신경망은 이미지 처리에 탁월한 성능을 보이는 신경망입니다. 이미지 처리를 하기 위해서 앞서 배운 다층 퍼셉트론을 사용할 수는 있지만 한계가 있었습니다. 예를 들어, 알파벳 손글씨를 분류하는 어떤 문제가 있다고 해봅시다. 아래의 그림은 알파벳 Y를 비교적 정자를 쓴 손글씨와 다소 휘갈겨 쓴 손글씨 두 개를 2차원 텐서인 행렬로 표현한 것입니다.

![](../Static/conv0.png)

사람이 보기에는 두 그림 모두 알파벳 Y로 손쉽게 판단이 가능하지만, 기계가 보기에는 각 픽셀마다 가진 값이 거의 상이하므로 완전히 다른 값을 가진 입력입니다. 그런데 이미지라는 것은 위와 같이 같은 대상이라도 휘어지거나, 이동되었거나, 방향이 뒤틀렸거나 등 다영한 변형이 존재합니다. 다층 퍼셉트론은 몇 가지 픽셀만 값이 달라져도 민감하게 예측에 영향을 받는다는 단점이 있습니다.

좀 더 구체적으로 보겠습니다. 위 손글씨를 다층 퍼셉트론으로 분류한다고 하면, 이미지를 1차원 텐서인 벡터로 변환하고 다층 퍼셉트론의 입력층으로 사용해야 합니다. 두번째 손글씨를 다층 퍼셉트론으로 분류하기 위해서 벡터로 바꾸면 다음과 같습니다.

![](../Static/conv1.png)   
1차원으로 변환된 결과는 사람이 보기에도 이게 원래 어떤 이미지였는지 알아보기가 어렵습니다. 이는 기계도 마찬가지입니다. 위와 같이 결과는 변환 전에 가지고 있던 공간적인 구조(Spatial structure) 정보가 유실된 상태입니다. 여기서 공간적인 구조 정보라는 것은 거리가 가까운 어떤 픽셀들끼리는 어떤 연관이 있고, 어떤 픽셀들끼리는 값이 비슷하거나 등을 포함하고 있습니다. 결국 이미지의 공간적인 구조 정보를 보존하면서 학습할 수 있는 방법이 필요해졌고, 이를 위해 사용하는 것이 `합성곱 신경망`입니다.

## 채널(Channel)

이미지 처리의 기본적인 용어인 채널에 대해서 간단히 정의하겠습니다.

기계는 글자나 이미지보다 숫자, 다시 말해, 텐서를 더 잘 처리할 수 있습니다. 이미지는 `(높이, 너비, 채널)`이라는 3차원 텐서입니다. 여기서 높이는 이미지의 세로 방향 픽셀 수, 너비는 이미지의 가로 방향 픽셀 수, 채널은 색 성분을 의미합니다. `흑백 이미지`는 채널 수가 1이며, 각 픽셀은 0부터 255 사이의 값을 가집니다. 아래는 28 x 28 픽셀의 손글씨 데이터를 보여줍니다.

![](../Static/conv2.png)   

위 손글씨 데이터는 흑백 이미지이므로 채널 수가 1임을 고려하면 (28 x 28 x 1)의 크기를 가지는 3차원 텐서입니다. 그렇다면 흑백이 아니라 우리가 통상적으로 접하게 되는 컬러 이미지는 어떨까요? `컬러 이미지`는 적색,녹색,청색 채널 수가 3개입니다.

![](../Static/conv3.png)   

하나의 픽셀은 세 가지 색깔, 즉 삼원색의 조합으로 이루어집니다. 만약, 높이가 28 너비가 28인 컬러 이미지가 있다면 이 이미지의 텐서는 (28 x 28 x 3)의 크기를 가지는 3차원 텐서입니다. 채널은 때로는 `깊이(Depth)`라고도 합니다. 이 경우 이미지는 `(높이, 너비, 깊이)`라는 3차원 텐서로 표현된다고 말할 수 있을 것입니다.

## 합성곱 연산(Convolution operation)
합성곱층은 합성곱 연산을 통해서 이미지의 특징을 추출하는 역할을 합니다. 우선, 합성곱 연산에 대해서 이해해봅시다. 합성곱은 영어로 컨볼루션이라고도 불리는데, `커널(Kernel)` 또는 `필터(Filter)`라는 n x m 크기의 행렬로 높이(Height) x 너비(Width) 크기의 이미지를 처음부터 끝까지 겹치며 훌ㅇ으면서 n x m 크기의 겹쳐지는 부분의 각 이미지와 커널의 원소의 값을 곱해서 모두 더한 값을 출력으로 하는 것을 말합니다. 이때, 이미지의 가장 왼쪽 위부터 가장 오른쪽까지 순차적으로 훑습니다.

* 커널(Kernel)은 일반적으로 3 x 3 또는 5 x 5 를 사용합니다.

예를 통해 이해해봅시다. 아래는 3x3 크기의 커널로 5x5의 이미지 행렬에 합성곱 연산을 수행하는 과정을 보여줍니다. 한 번의 연산을 `1스텝(Step)`이라고 하였을 때, 합성곱 연산의 네번째 스텝까지 이미지와 식으로 표현해봤습니다.

#### 1.첫번째 스텝
![](../Static/conv4.png)   
(1×1) + (2×0) + (3×1) + (2×1) + (1×0) + (0×1) + (3×0) + (0×1) + (1×0) = 6

#### 2.두번째 스텝
![](../Static/conv5.png)   
(2×1) + (3×0) + (4×1) + (1×1) + (0×0) + (1×1) + (0×0) + (1×1) + (1×0) = 9

#### 3.세번째 스텝
![](../Static/conv6.png)   
(3×1) + (4×0) + (5×1) + (0×1) + (1×0) + (2×1) + (1×0) + (1×1) + (0×0) = 11

#### 4.네번째 스텝
![](../Static/conv7.png)   
(2×1) + (1×0) + (0×1) + (3×1) + (0×0) + (1×1) + (1×0) + (4×1) + (1×0) = 10

위 연산을 총 9번의 스텝까지 마쳤다고 가정하였을 때, 최종 결과는 아래와 같습니다.   
![](../Static/conv8.png)   


위와 같이 입력으로부터 커널을 사용하여 합성곱 연산을 통해 나온 결과를 `특성 맵(Feature map)`이라고 합니다.

위의 예제에서는 커널의 크기가 3 x 3 이였지만, 커널의 크기는 사용자가 정할 수 있습니다. 또한 커널의 이동 범위가 위의 예제에서는 한 칸이었지만, 이 또한 사용자가 정할 수 있습니다.
이러한 이동 범위를 `스트라이드(stride)`라고 합니다.

아래의 예제는 스트라이드가 2일 경우에 5 x 5 이미지에 합성곱 연산을 수행하는 3 x 3 커널의 움직임을 보여줍니다. 최종적으로 2 x 2의 크기의 특성 맵을 얻습니다.
![](../Static/conv9.png)